In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os, time, shutil
from PIL import Image
from time import gmtime, strftime
import csv
import cv2
import glob
import serial,sys
import signal

from IPython.display import clear_output
from IPython import display

%matplotlib inline


In [4]:
def get_cameras():
    arr = []
    for index in range(10):
        print(index)
        cap = cv2.VideoCapture(index)
        if not cap.read()[0]:
            continue
        else:
            arr.append(index)
        cap.release()
    return arr

# Init cameras
## Get available cameras

In [5]:
cams_idxs = get_cameras()
cams_idxs

0
1
2
3
4
5
6
7
8
9


[0, 2, 3]

In [8]:

#
# Attach interrupt handler to release cameras after sigterm
def release_caps(caps):
    for cap in caps:
        try:
            cap.release()
            print("Released camera")
        except Exception as e:
            print("Error",e)
        
def gen_int_handler(caps):
    def int_handler(signum, frame):
        print("Got sigterm")
        release_caps(caps)
        raise  Exception('Interrupt') 
        #cv2.destroyAllWindows()
    return int_handler

def init_caps(idx_list):
    print(f"Init cameras {idx_list}")
    caps = [cv2.VideoCapture(i) for i in (idx_list) ]

    # Set exposition time-------
    for cap in caps:
        if not cap.isOpened():
            print("Failed to init")
        cap.set(cv2.CAP_PROP_AUTO_EXPOSURE, 1)
        # 10 was found to be the smallest value
        cap.set(cv2.CAP_PROP_EXPOSURE, 10)
        print("Param::auto exp?",cap.get(cv2.CAP_PROP_AUTO_EXPOSURE))
        print("Param::exp time", cap.get(cv2.CAP_PROP_EXPOSURE))

    signal.signal(signal.SIGINT, gen_int_handler(caps))
    return caps


In [9]:

try:    #do not init if caps are defined
    release_caps(caps)
except NameError as e:
    pass

caps = init_caps(cams_idxs)
#caps = init_caps([0,2])

caps

Released camera
Released camera
Released camera
Init cameras [0, 2, 3]
Param::auto exp? 0.75
Param::exp time -0.00040016006402561027
Param::auto exp? 0.75
Param::exp time 0.19965870307167236
Param::auto exp? 0.75
Param::exp time -0.00040016006402561027


[<VideoCapture 0x7f3194b17450>,
 <VideoCapture 0x7f3194b17410>,
 <VideoCapture 0x7f3198678850>]

In [10]:

class CameraNoneException(Exception):
    def __init___(self,dErrorArguments):
        Exception.__init__(self,"Camera read None {0}".format(dErrArguments))
        self.dErrorArguments = dErrorArguements
        
        
def read_cameras(caps, idxs, retry_cnt=0):
    results = []
    for camera_idx, cap in zip(idxs,caps):
        print(f"Reading {camera_idx}")
        if cap.isOpened():
            ret, img = cap.read()
        else:
            print("Camera is closed.")
            ret = None
        if not ret:
            print(f"Camera {camera_idx} returned None")

            if retry_cnt<3:
                release_caps(caps)
                time.sleep(0.1)
                print(f"Trying to connect to {camera_idx} only")
                cap_retry = init_caps([camera_idx])
                time.sleep(0.1)
                res_retry = read_cameras(cap_retry, [camera_idx], retry_cnt=retry_cnt+1)
                results += res_retry
            else:
                print(f"!>>\nCamera {camera_idx} is not working at all")
                continue
        else:
            results.append(img)
    return results

results = read_cameras(caps, cams_idxs)
print(len(results))


Reading 0
Reading 2
Camera 2 returned None
Released camera
Released camera
Released camera
Trying to connect to 2 only
Init cameras [2]
Failed to init
Param::auto exp? 0.0
Param::exp time 0.0
Reading 2
Camera is closed.
Camera 2 returned None
Released camera
Trying to connect to 2 only
Init cameras [2]
Failed to init
Param::auto exp? 0.0
Param::exp time 0.0
Reading 2
Camera is closed.
Camera 2 returned None
Released camera
Trying to connect to 2 only
Init cameras [2]
Failed to init
Param::auto exp? 0.0
Param::exp time 0.0
Reading 2
Camera is closed.
Camera 2 returned None
!>>
Camera 2 is not working at all
Reading 3
Camera is closed.
Camera 3 returned None
Released camera
Released camera
Released camera
Trying to connect to 3 only
Init cameras [3]
Param::auto exp? 0.75
Param::exp time -0.00040016006402561027
Reading 3
2


In [7]:
while True:
    for camera_idx, cap in zip(cams_idxs,caps):
        ret, img = cap.read()
        if not ret:
            print(f"Camera {camera_idx} returned None")
            print("Releasing all cameras and trying this camera only")
    if (tries==retry_cnt):
        print("Camera %i is not working. Giving up."%num)
        print()
        break
    time.sleep(retry_sec)


TypeError: VideoCapture.get() missing required argument 'propId' (pos 1)

In [1]:
import cv2
cap0 = cv2.VideoCapture(0)
cap0.set(3,160)
cap0.set(4,120)
cap1 = cv2.VideoCapture(2)
cap1.set(3,160)
cap1.set(4,120)
ret0, frame0 = cap0.read()
assert ret0 # succeeds
ret1, frame1 = cap1.read()
assert ret1 # fails?!

AssertionError: 

# Threaded camera reading

In [19]:
import cv2
import threading
import time

class camThread(threading.Thread):
    def __init__(self, img_array, camID, cap):
        threading.Thread.__init__(self)
        self.camID = camID
        self.cap = cap
        self.array = img_array
    def run(self):t
        print ("Starting %s"% self.camID)
        if self.cap.isOpened():  # try to get the first frame
            rval, frame = self.cap.read()
            time.sleep(0)
            if not rval:
                print("fail", self.camID)
                frames = read

        #pic = take_pic(self.camID)
            self.array.append(frame)
        else:
            print("fail")

def take_pic(camID):
    cam = cv2.VideoCapture(camID)
    if cam.isOpened():  # try to get the first frame
        time.sleep(0)
        rval, frame = cam.read()
        time.sleep(0)
        print('read')
        return frame
    else:
        rval = False

# Create two threads as follows
a = []
b = []

thread1 = camThread(a, 0, caps[0])
thread2 = camThread(b, 2, caps[1])
thread1.start()
thread2.start()

Starting 0
Starting 2
fail 2


In [10]:
from cameras.threaded_cam import Camera


In [11]:
cid = get_cameras()
cid

0
1
2
3
4
5
6
7
8
9


[0, 2, 3]

In [12]:
f = [Camera(i) for i in cid]
ps  = [c.start() for c in f]


trying to open Camera0
trying to open Camera2
trying to open Camera3
trying to open Camera0


In [14]:
f[0].buffer.qsize

<bound method Queue.qsize of <queue.Queue object at 0x7fd220ea3e10>>

In [6]:
import cvpubsubs.webcam_pub as w
from cvpubsubs.window_sub import SubscriberWindows

t1 = w.VideoHandlerThread(0)
t2 = w.VideoHandlerThread(2)
t3 = w.VideoHandlerThread(3)

t1.start()
t2.start()
t3.start()

SubscriberWindows(window_names=['cammy', 'cammy2','cam3'],
              video_sources=[0,2,3]
              ).loop()

t1.join()
t2.join()
t3.join()

KeyboardInterrupt: 